In [2]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt

In [3]:
def stop_residual(A, b, x, x_new):
    return np.max(np.abs(np.array(np.matmul(A, x)) - np.array(b)))


def stop_incremental(A, b, x, x_new):
    return np.max(np.abs(x - x_new))

In [4]:
def avg_err(x, x_new):
    return np.average(np.abs(np.array(x) - np.array(x_new)))

In [5]:
def get_matrix(n):
    A = np.zeros((n,n))
    for i in range(n):
        A[i, i] = 10
        if i != 0:
            A[i, i-1] = 3 / (i + 1)

        for j in range(n):
            if j > i:
                A[i, j] = np.power(-1, j+1) * (3 / (j + 1))

    return A

In [ ]:
def _jacobi_x(A, b, x):
    n = len(A)
    new_x = np.zeros(n)
    for i in range(n):
        s = 0
        for j in range(n):
            if j != i:
                s += A[i, j] * x[j]
        new_x[i] = (b[i] - s) / A[i, i]
    return new_x


def jacobi(A, b, x, eps, stop, max_iter=1000):
    n = len(b)
    x_new = x.copy()

    start = time.time()
    iterations = 1
    while True:
        for i in range(n):
            row_values = A[i, :i].tolist() + A[i, i + 1:].tolist()
            row_sum = np.dot(np.array(row_values), np.concatenate((x[:i], x[i + 1:])))
            x_new[i] = (b[i] - row_sum) / A[i, i]


        if stop(x, x_new, A, b) < eps:
            t = time.time() - start
            return x_new, iterations, t

        if iterations > max_iter:
            t = time.time() - start
            return x_new, iterations, t


        iterations += 1
        x = x_new.copy()


def get_random_x1(n):
    return [random.choice([-1, 1]) for _ in range(n)]

def get_random_x2(n):
    return [random.choice([-1, 1]) * random.randint(80, 120) for _ in range(n)]

In [ ]:
def plot(comps, title, x_label, y_label):
    N = [3, 5, 10, 15, 20, 25, 50, 75, 100, 200, 300, 400, 500]
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.plot(N, comps[0], label="kryterium rezydualne, wektor typu a")
    plt.plot(N, comps[1], label="kryterium przyrostowe, wektor typu a")
    plt.plot(N, comps[2], label="kryterium rezydualne, wektor typu b")
    plt.plot(N, comps[3], label="kryterium przyrostowe, wektor typu b")
    plt.legend()

    plt.show()

In [7]:
X = get_random_x1(10000)
X_start = get_random_x2(10000)
N = [3, 5, 10, 15, 20, 25, 50]

# n_r, n_i, f_r, f_i
comp_times = [[], [], [], []]
comp_results = [[], [], [], []]
comp_iterations = [[], [], [], []]

In [8]:
def test_n():
    N = [3, 5, 10, 15, 20, 25, 50, 75, 100, 200, 300, 400, 500]
    results = [
        ["n", "liczba iteracji", "-",  "błąd średni", "-", "czas (s)", "-"],
        ["-", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe"]
    ]
    eps = 10e-10

    for n in N:
        matrix = get_matrix(n)
        x = get_random_x1(n)
        b = np.matmul(matrix, x)
        r_x, r_i, r_t = jacobi(matrix, b, get_random_x1(n), eps, stop_residual)
        i_x, i_i, i_t = jacobi(matrix, b, get_random_x1(n), eps, stop_incremental)

        comp_times[0].append(r_t)
        comp_times[1].append(i_t)

        comp_results[0].append(avg_err(x, r_x))
        comp_results[1].append(avg_err(x, i_x))

        comp_iterations[0].append(r_i)
        comp_iterations[1].append(i_i)

        results.append([str(n), str(r_i), str(i_i), f"{avg_err(x, r_x):.5e}", f"{avg_err(x, i_x):.5e}", f"{r_t:.5f}", f"{i_t:.5f}"])

    for row in results:
        print("\t".join(row))

In [12]:
def test_x():
    N = [3, 5, 10, 15, 20, 25, 50, 75, 100, 200, 300, 400, 500]
    results = [
        ["n", "liczba iteracji", "-",  "błąd średni", "-", "czas (s)", "-"],
        ["-", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe"]
    ]
    eps = 10e-10

    for n in N:
        matrix = get_matrix(n)
        x = get_random_x1(n)
        b = np.matmul(matrix, x)
        r_x, r_i, r_t = jacobi(matrix, b, get_random_x2(n), eps, stop_residual)
        i_x, i_i, i_t = jacobi(matrix, b, get_random_x2(n), eps, stop_incremental)

        comp_times[2].append(r_t)
        comp_times[3].append(i_t)

        comp_results[2].append(avg_err(x, r_x))
        comp_results[3].append(avg_err(x, i_x))

        comp_iterations[2].append(r_i)
        comp_iterations[3].append(i_i)

        results.append([str(n), str(r_i), str(i_i), f"{avg_err(x, r_x):.5e}", f"{avg_err(x, i_x):.5e}", f"{r_t:.5f}", f"{i_t:.5f}"])

    for row in results:
        print("\t".join(row))



In [ ]:
def test_eps():
    N = 100
    results = [
        ["eps", "liczba iteracji", "-",  "błąd średni", "-", "czas (s)", "-"],
        ["-", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe", "rezydualne", "przyrostowe"]
    ]

    EPS = [10**(-i) for i in range(1, 11)] + [10**-15]


    for eps in EPS:
        matrix = get_matrix(N)
        x = get_random_x1(N)
        b = np.matmul(matrix, x)
        r_x, r_i, r_t = jacobi(matrix, b, get_random_x1(N), eps, stop_residual)
        i_x, i_i, i_t = jacobi(matrix, b, get_random_x1(N), eps, stop_incremental)


        results.append([f"{eps:.0e}", str(r_i), str(i_i), f"{avg_err(x, r_x):.5e}", f"{avg_err(x, i_x):.5e}", f"{r_t:.5f}", f"{i_t:.5f}"])

    for row in results:
        print("\t".join(row))


In [ ]:
test_n()

In [ ]:
test_x()

In [ ]:
test_eps()

In [ ]:
plot(comp_times, "Czasy wykonywania programu", "n", "czas (s)")
plot(comp_iterations, "Liczba iteracji", "n", "liczba iteracji")
plot(comp_results, "Błędy średnie", "n", "błąd średni")